In [4]:
import os
import pandas as pd
import numpy as np
import re
import utils

pd.set_option('display.float_format', '{:.2f}'.format)

# Configuración de rutas
ruta = os.getcwd()
ruta_data = os.path.abspath(os.path.join(ruta, '..', 'data'))
file_obligaciones = os.path.join(ruta_data, 'datos_crudos', 'obligaciones_clientes.xlsx')
file_tasas = os.path.join(ruta_data, 'datos_crudos', 'tasas_productos.xlsx')
output_file_obligaciones = os.path.join(ruta_data, 'datos_refinados', 'obligacionesConTasas.csv')
output_file_clientes = os.path.join(ruta_data, 'datos_refinados', 'clientesConMultiplesProductos.csv')

# Verificar la existencia de los archivos necesarios
if not os.path.exists(file_obligaciones) or not os.path.exists(file_tasas):
    raise FileNotFoundError(f"Los archivos obligatorios no se encontraron en la ruta: {ruta_data}")

# Leer archivos Excel
df_obligaciones = pd.read_excel(file_obligaciones, sheet_name=0, dtype={'cod_segm_tasa': str, 'cod_subsegm_tasa': str, 'radicado' : str, 'num_documento' : str})
df_tasas = pd.read_excel(file_tasas, sheet_name=0, dtype={'cod_segmento': str, 'cod_subsegmento': str})

df_obligaciones['plazo'] = df_obligaciones['plazo'].replace('None', pd.NA).astype('Int64')

# Mapeo general de variables 
# Mapeo1 = utils.func_mapeo_de_variables(df_obligaciones)
# display(Mapeo1)

# Mapeo2 = utils.func_mapeo_de_variables(df_tasas)
# display(Mapeo2)

# Función para obtener el nombre del producto
def func_nombre_producto(id_producto):
    try:
        if isinstance(id_producto, str):
            # Reemplazar guiones largos por guiones cortos
            id_producto = id_producto.replace('–', '-')
            parts = [part.strip() for part in id_producto.split('-')]
            if len(parts) >= 2:
                last_part = parts[-1]
                match = re.search(r'[a-zA-Z_]+', last_part)
                return match.group(0).lower() if match else 'desconocido'
            else:
                return 'desconocido'
        else:
            return 'desconocido'
    except Exception as e:
        print(f"Error procesando id_producto: {id_producto} - {str(e)}")
        return 'desconocido'

# Agregar la columna 'nombre_producto' al DataFrame
df_obligaciones['nombre_producto'] = df_obligaciones['id_producto'].apply(func_nombre_producto)

# Mapeo de productos a columnas de tasas
product_to_tasa = {
    'cartera': 'tasa_cartera',
    'operacion_especifica': 'tasa_operacion_especifica',
    'hipotecario': 'tasa_hipotecario',
    'leasing': 'tasa_leasing',
    'sufi': 'tasa_sufi',
    'factoring': 'tasa_factoring',
    'tarjeta': 'tasa_tarjeta'
}

# Función para asignar la tasa correcta
def func_asignar_tasa(row, tasas_df):
    producto = row['nombre_producto']
    if producto in product_to_tasa:
        tasa_column = product_to_tasa[producto]
        matching_tasa = tasas_df[
            (tasas_df['cod_segmento'] == row['cod_segm_tasa']) &
            (tasas_df['cod_subsegmento'] == row['cod_subsegm_tasa']) &
            (tasas_df['calificacion_riesgos'] == row['cal_interna_tasa'])
        ]
        if not matching_tasa.empty:
            return matching_tasa[tasa_column].values[0]
    return np.nan

# Asignar las tasas a cada producto en el DataFrame
df_obligaciones['tasa_asignada'] = df_obligaciones.apply(lambda row: func_asignar_tasa(row, df_tasas), axis=1)

# Función para calcular la tasa efectiva
def func_calcular_tasa_efectiva(row):
    t = row['tasa_asignada']
    periodicidad = row['periodicidad']
    
    if pd.isna(t) or pd.isna(periodicidad):
        return np.nan
    
    periodicidad_map = {'MENSUAL': 1, 'BIMENSUAL': 2, 'TRIMESTRAL': 3, 'SEMESTRAL': 6, 'ANUAL': 12}
    n = 12 / periodicidad_map.get(periodicidad, 1)
    
    return (1 + t) ** (1 / n) - 1

# Calcular la tasa efectiva
df_obligaciones['tasa_efectiva'] = df_obligaciones.apply(func_calcular_tasa_efectiva, axis=1)

# Calcular el valor final
df_obligaciones['valor_final'] = df_obligaciones['valor_inicial'] *  df_obligaciones['tasa_efectiva']

# Filtrar que si tengan saldo en deuda
df_obligaciones_por_cliente = df_obligaciones[df_obligaciones['saldo_deuda'] > 0]
# Agrupo por los que indican hace la llave
df_obligaciones_por_cliente = df_obligaciones_por_cliente.groupby(['radicado', 'num_documento', 'id_producto', 'valor_inicial', 'fecha_desembolso'], as_index = False   ).agg({'valor_final': 'sum'})
# Cuenta los productos por cliente
df_obligaciones_por_cliente = df_obligaciones_por_cliente.groupby(['num_documento'], as_index = False).agg({'valor_final': 'sum','id_producto': 'count'})
# Filtrar por los que tienen 2 o más productos
df_clientes_con_mas_2_obligaciones = df_obligaciones_por_cliente[df_obligaciones_por_cliente['id_producto'] >= 2]
df_clientes_con_mas_2_obligaciones.rename({'id_producto' : 'cantidad_obligaciones'}, axis = 1, inplace = True)

# Guardar los resultados en la carpeta 'data'
df_obligaciones.to_csv(output_file_obligaciones, index=False, sep = "|")
df_clientes_con_mas_2_obligaciones.to_csv(output_file_clientes, index=False, sep = "|")

print(f"Procesamiento completado. Los resultados se han guardado en:\n- {output_file_obligaciones}\n- {output_file_clientes}")

Procesamiento completado. Los resultados se han guardado en:
- c:\Users\saarias\OneDrive - CUEROS VELEZ S A S\Documentos\Descargas\Prueba_tecnica\Prueba_tecnica_ingeniero_datos\data\datos_refinados\obligacionesConTasas.csv
- c:\Users\saarias\OneDrive - CUEROS VELEZ S A S\Documentos\Descargas\Prueba_tecnica\Prueba_tecnica_ingeniero_datos\data\datos_refinados\clientesConMultiplesProductos.csv


C:\Users\saarias\AppData\Local\Temp\ipykernel_5028\89367396.py:111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clientes_con_mas_2_obligaciones.rename({'id_producto' : 'cantidad_obligaciones'}, axis = 1, inplace = True)


### Clientes con > 2 obligacioens

In [5]:
df_clientes_con_mas_2_obligaciones.sort_values(['cantidad_obligaciones'], ascending = False)

,num_documento,valor_final,cantidad_obligaciones
286,996040778,465784.83,55
69,1080618793,239000.82,32
289,996065497,178293.03,32
99,1081045732,238627.22,30
258,960246199,507875.36,30
...,...,...,...
81,1080762446,4544.93,2
247,91540839,2052.72,2
248,9238900,3178.43,2
77,1080709552,76948.46,2
